# <center>Network Science</center>

## <center/>Course Project #2

### <center>Student: *Salnikov Alexander*</center>

#### <hr /> General Information

**Due Date:** 17.06.2016 23:59 <br \>
**Late submission policy:** no grade <br \>


Please send your reports to <mailto:network.hse.2016@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2016] *Alexander* *Salnikov* Project*2***

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

## Description

### Data

As a dataset to analyse you can choose one option in the following list:
1. Real Dataset (can be found [here](http://snap.stanford.edu/) or [here](http://konect.uni-koblenz.de/networks/))
2. Generated Dataset. Use more complex structure rather than just a simple ER model. For instance, you may consider multilevel network, where on the lower level you have several Watts-Strogatz graphs and on the upper level these graphs are respesented as randomly connected nodes.
3. Your data mined from Social Networks, Twitter, LiveJournal e.t.c.

**The order of your dataset should be no less than $10^4$ nodes**

### Models

Consider one of the following models:
1. SIR-based (or another with more than 3 letters) epidemic model
2. Independent Cascade Model
3. Linear Threshold Model

### Tasks

#### Network Descriptive Analysis

Provide information on your netowork: Source, Descriptive Statistics, Visualization

#### Main Task for model (1)

You are in charge of leading the vaccination campaign against some outbroken nonlethal disease. You have options to vactinate or provide medical treatment to infected ones. However, everything has its costs:
* Vaccination of a node costs $500 \$$ and make it immune to the disease all life-long. Unfortunately, you can help this way only to no more than $10\%$ of your population
* Medical Treatment costs $120\$$ per day of illness period, which in turn may take from $3$ to $7$ days

Your task is to implement the simulation model, propose some vaccination strategies and compare them.

#### Main Task for models (2-3)

You are running the marketing campaign for brand new pocket device. Initially you can sign contracts with a few people to advertize your gadget among their neigbours. The more "famous" person you are picking the greater price appears in the contract.
* Contract cost can be calculated as $300 \$ \times \text{NN}(i)$, where $\text{NN}(i)$ is size of the neigbourhood of the person $i$.
* You earn $250\$$ per each affected person

Your task is to maximize your influence and maximize profit of your campaign

In [134]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


/Users/insider/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension(s)as a python packages.
  "as a python packages.", UserWarning)


In [42]:
import numpy as np
import networkx as nx

In [43]:
G = nx.read_edgelist("out.ego-twitter", delimiter=" ",comments="%",nodetype=int)

In [44]:
A = np.array(nx.adjacency_matrix(G).todense())
n = A.shape[0]

In [135]:
theta = np.random.random_sample(n)

time: 2.29 ms


In [46]:
deg = np.sum(A,axis=0, dtype=float)

In [47]:
def InfluenceProp(A, initActive, theta, itersNum = np.inf):
    i = 1 # iteration
    resActive = initActive.copy()    
    while i < itersNum:
        i+=1

        # currently inactive nodes
        inactiveId = np.where(resActive == 0)[0]    
        # activated nodes
        idx = np.sum(A[np.ix_(resActive==1, resActive==0)], axis=0) / deg[resActive==0] > theta[inactiveId]
        if np.any(idx):
            resActive[inactiveId[idx]] = 1
        else:
            break

    return resActive

In [62]:
def LenInfluenceProp(A, initActive, theta, itersNum = np.inf):
    resultActive = InfluenceProp(A, initActive, theta, itersNum=itersNum)
    return len(np.where(resultActive==True)[0])

In [132]:
def greedy(A, theta, itersNum = np.inf):
    n = A.shape[0]
    InitActive = np.zeros((n,), dtype=bool)
    marginal_gain = np.zeros((n,2), dtype=int)
    marginal_gain[:,0] = range(n)
    result = []
    
    step = 0
    while True:
        current_influence = LenInfluenceProp(A, InitActive, theta, itersNum=itersNum)
        # calc marginal gain for all nodes on first step
        top_to_recalculate = range(n) if step == 0 else range(10) 
        for i in top_to_recalculate:
            tempInitActive = InitActive.copy()
            tempInitActive[marginal_gain[i,0]] = 1
            marginal_gain[i,1] = LenInfluenceProp(A, tempInitActive, theta, itersNum=itersNum)- current_influence
        
        marginal_gain = marginal_gain[marginal_gain[:,1].argsort()][::-1]
        InitActive[marginal_gain[0,0]] = 1
        result.append(marginal_gain[0,0])
        
#         if marginal_gain[0,1] == 1:
#             break          
        if step == 300 or marginal_gain[0,1]==1 or marginal_gain[0,1]==0:
            break

        marginal_gain[0,1] = -1 # make sure this node will never be on top anymore
        marginal_gain = marginal_gain[marginal_gain[:,1].argsort()][::-1]
        

        step = step +1


    return result
        
        
        
        
#     step = 1
#     for Id in currentInitInactiveId:
#         tempInitActive = currentInitActive.copy()
#         tempInitActive[Id] = 1
#         resultActive = InfluenceProp(A, tempInitActive, theta, deg, itersNum=itersNum)
#         ordered_list_of_marginal_gain[Id,1] = len(np.where(resultActive==True)[0])

#     ordered_list_of_marginal_gain = ordered_list_of_marginal_gain[ordered_list_of_marginal_gain[:,1].argsort()]
    
    
#     return ordered_list_of_marginal_gain
    

In [155]:
temp = greedy(A, theta)

KeyboardInterrupt: 

time: 6h 40min 53s


In [ ]:
print(temp)

In [150]:
idx = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000]
initActive = np.zeros((n,), dtype=bool)
initActive[idx] = 1

time: 22.8 ms


In [153]:
resultActive = InfluenceProp(A, initActive, theta)
print(len(np.where(resultActive==True)[0]))

3008
time: 3.68 s


In [26]:
    ordered_list_of_nodes = np.zeros((n,2), dtype=int)
    ordered_list_of_nodes[:,0] = range(n)
    print(ordered_list_of_nodes[:,0])
    #    ordered_list_of_nodes[ordered_list_of_nodes[:,1].argsort()]

[    0     1     2 ..., 23367 23368 23369]


In [156]:
outcome = np.array([])
np.ix

{}

Chosen dataset is http://konect.uni-koblenz.de/networks/ego-twitter